## <span style="font-family:Georgia, serif;">**Twitter Sentiment Analysis** :Understanding Emotions in Tweets about Apple and Google products.</span>

![alt text](twits.jpg "Title")

## Overview

## Business Understanding

**Business Problem**: Using Sentiment Analysis to Improve Apple and Google Product Marketing Strategies 

The introduction of social media has completely changed how businesses interact with their consumers and the general public in today's connected society. While the digital age offers limitless possibilities for marketing and brand development, it also brings its own set of difficulties. One of these difficulties is the inability of enterprises to precisely gauge public opinion and feelings towards their goods or services.

In the age of social media, organizations are acutely aware of the need to harness the wealth of sentiment and emotion data available on these platforms. However, they often struggle to do so effectively, given the unprecedented speed, diversity, and complexity of social media communication. The dynamic nature of the medium, the diverse and contextual language used, the rapid increase of emojis and visual content, the volume of noise, and ethical concerns all contribute to the challenge of gauging public sentiment and emotions. 

To overcome these challenges, organizations must invest in advanced sentiment analysis tools and technologies, develop cultural and linguistic expertise, and strike a balance between data-driven insights and ethical considerations. By doing so, they can unlock the valuable insights hidden within the social media storm and use them to inform strategic decisions, enhance products and services, and build stronger connections with their audience in this rapidly evolving digital landscape.




## Data understanding

## Data Preparation

In [60]:
import pandas as pd
import re
import io
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

def remove_non_utf8(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

with open('data/judge_1377884607_tweet_product_company.csv', 'r', encoding='utf-8') as file:
    cleaned_text = remove_non_utf8(file.read())

df = pd.read_csv(io.StringIO(cleaned_text))
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [61]:
column_name_mapping = {'is_there_an_emotion_directed_at_a_brand_or_product': 'Sentiment'}
# Rename the columns using the .rename() method
df.rename(columns=column_name_mapping, inplace=True)


,tweet_text,emotion_in_tweet_is_directed_at,Sentiment
1337,"Conan doc, CDR Radio, Android party, NPR shind...",NaN,No emotion toward brand or product
7432,Hot Pot from Google for mobile maps is a 'very...,Other Google product or service,Positive emotion
395,If you're coming to #SXSW be sure to download ...,iPad or iPhone App,Positive emotion
6230,RT @mention Questioner at #sxsw just said &quo...,Google,Positive emotion
27,someone started an #austin @PartnerHub group i...,Other Google product or service,Positive emotion


In [62]:
df['emotion_in_tweet_is_directed_at'].fillna('N/A', inplace=True)
df['tweet_text'].fillna('N/A', inplace=True)

In [63]:
def assign_brand(phrase):
    if 'iPad' in phrase or 'iPhone' in phrase :
        return 'Apple'
    elif 'Other Apple product or service' in phrase or 'Apple' in phrase:
        return 'Apple' 
    elif 'iPad or iPhone App' in phrase:
        return 'Apple'       
    elif 'Google' in phrase or 'Other Google product or service' in phrase:
        return 'Google'
    elif 'Android App' in phrase or 'Android' in phrase:
        return 'Android'
    else:
        return 'N/A'

df['brand'] = df['emotion_in_tweet_is_directed_at'].apply(assign_brand)

,tweet_text,emotion_in_tweet_is_directed_at,Sentiment,brand
5109,"RT @mention Happy Woman's Day! Make love, not...",N/A,No emotion toward brand or product,N/A
3183,"#Apple, #Google, #Intel and Others Go Gaga for...",N/A,I can't tell,N/A
7512,"Yes, the rumors are true. Come play with @ment...",N/A,Positive emotion,N/A
6190,RT @mention Our #sxsw app is now on #android :...,Android App,Positive emotion,Android
8417,Thank heavens for nice travelers. Got a iPhone...,iPhone,Positive emotion,Apple


In [64]:
import nltk
import re
from nltk.tokenize import word_tokenize,TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download('punkt')
# nltk.download('stopwords')


def clean_and_preprocess_text(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove mentions (words starting with '@') and URLs
    tokens = [token for token in tokens if not token.startswith('@') and not token.startswith('http')]
    # Remove punctuation and numbers using regular expressions
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Apply stemming using the Porter Stemmer
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    cleaned_text = ' '.join(stemmed_tokens) 
    return cleaned_text

In [65]:
df['tweet_text'][4]

"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"

In [66]:
clean_and_preprocess_text(df['tweet_text'][4])

'great stuff fri sxsw  marissa mayer  googl   tim oreilli  tech book  confer   matt mullenweg  wordpress '

In [67]:
df['processed_text'] = df['tweet_text'].map(clean_and_preprocess_text)

,tweet_text,emotion_in_tweet_is_directed_at,Sentiment,brand,processed_text
5873,RT @mention If you were able to afford to atte...,N/A,No emotion toward brand or product,N/A,rt abl afford attend sxsw buy ipad today cons...
2097,iPad giveaway for using @mention at #sxsw {link},iPad,Positive emotion,Apple,ipad giveaway use sxsw link
8704,At &quot;Your Mom Has an iPad&quot; session #y...,N/A,No emotion toward brand or product,N/A,mom ipad session yourmom sxsw
2237,#sxsw #enchantment: @mention &quot;It's like s...,N/A,No emotion toward brand or product,N/A,sxsw enchant like say pleas dont regist go...
3392,@mention I want that iPad case! #sxswbuffalo #...,iPad,Positive emotion,Apple,want ipad case sxswbuffalo sxsw


In [68]:
# Define a mapping dictionary
sentiment_mapping = {'No emotion toward brand or product': 2.0,
                  'Positive emotion': 1.0, 
                  'Negative emotion': 0.0,
                  'I can\'t tell': 2.0}

# Use the .map() method to map values in column 'A' to new values
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)
df.sample(5)

,tweet_text,emotion_in_tweet_is_directed_at,Sentiment,brand,processed_text
4041,You finally get everyone to buy in to Facebook...,Other Google product or service,0.0,Google,final get everyon buy facebook googl introduc ...
4390,Interesting debate on jcpenney tactics on site...,N/A,2.0,N/A,interest debat jcpenney tactic site show irrel...
2280,"After... Come visit Milyoni we are at Booth, 1...",N/A,2.0,N/A,come visit milyoni booth right big googl si...
5160,RT @mention Android may be gaining market shar...,N/A,2.0,N/A,rt android may gain market share youd never k...
3600,And @mention opens with some tech talk on the ...,N/A,2.0,N/A,open tech talk appl store whore pc right crow...


In [69]:
from sklearn.model_selection import train_test_split
#creating new df where sentiment is either positive or negative
bi_tar = df[(df['Sentiment'] == 0)| (df['Sentiment'] == 1)]

X = bi_tar['processed_text']
y = bi_tar['Sentiment']

X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
multi_tar = df.copy()
X = multi_tar['processed_text']
y = multi_tar['Sentiment']

y_dummies = pd.get_dummies(y)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_dummies, test_size=0.2, random_state=42)

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

def csr_Tfid_vect(X_train,X_test):
    vectorizer = TfidfVectorizer()
    tf_idf_train = vectorizer.fit_transform(X_train)
    tf_idf_test = vectorizer.transform(X_test)

    tf_idf_train = csr_matrix(tf_idf_train)
    tf_idf_test = csr_matrix(tf_idf_test)

    return tf_idf_train,tf_idf_test

In [72]:
X_tf_idf_train_bi,X_tf_idf_test_bi = csr_Tfid_vect(X_train_bi,X_test_bi)

X_tf_idf_train_multi,X_tf_idf_test_multi = csr_Tfid_vect(X_train_multi,X_test_multi)

In [73]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.preprocessing import text, sequence

def token_seq(feature):
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(list(feature))
    list_tokenized = tokenizer.texts_to_sequences(feature)
    seq = sequence.pad_sequences(list_tokenized, maxlen=100)
    return seq

## Text Analysis

In [89]:
from  nltk import FreqDist
import string

big_sentence = ' '.join(df['tweet_text'])
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tweets_raw = nltk.regexp_tokenize(big_sentence, pattern)
tweets_raw = [word.lower() for word in tweets_raw]
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
tweets_raw_stopped = [word for word in tweets_raw if word not in stopwords_list]
tweets_freqdist = FreqDist(tweets_raw_stopped)
tweets_freqdist.most_common(10)

[('sxsw', 9256),
 ('mention', 6950),
 ('link', 4122),
 ('rt', 2959),
 ('ipad', 2882),
 ('google', 2373),
 ('apple', 2124),
 ('quot', 1621),
 ('iphone', 1525),
 ('store', 1420)]

In [91]:
total_word_count = sum(tweets_freqdist.values())
tweets_freqdist_top_10 = tweets_freqdist.most_common(10)
print(f'{"Word":10} Normalized Frequency')
for word in tweets_freqdist_top_10:
    normalized_frequency = word[1] / total_word_count
    print(f'{word[0]:10} {normalized_frequency:^20.4}')

Word       Normalized Frequency
sxsw              0.0858       
mention          0.06442       
link             0.03821       
rt               0.02743       
ipad             0.02671       
google            0.022        
apple            0.01969       
quot             0.01503       
iphone           0.01414       
store            0.01316       


In [92]:
from nltk.collocations import BigramCollocationFinder

bigram_measures = nltk.collocations.BigramAssocMeasures()
tweets_finder = BigramCollocationFinder.from_words(tweets_raw_stopped)
tweets_scored = tweets_finder.score_ngrams(bigram_measures.raw_freq)
tweets_scored[:15]

[(('rt', 'mention'), 0.02666802617674867),
 (('sxsw', 'link'), 0.008527835968929014),
 (('link', 'sxsw'), 0.007656513598190615),
 (('sxsw', 'rt'), 0.006275374946701025),
 (('mention', 'mention'), 0.005728481118258838),
 (('mention', 'sxsw'), 0.005478207671344618),
 (('apple', 'store'), 0.005348436254426132),
 (('sxsw', 'mention'), 0.004755195491370201),
 (('link', 'rt'), 0.004718117943679205),
 (('mention', 'google'), 0.004356611853691997),
 (('social', 'network'), 0.0040970690198550265),
 (('new', 'social'), 0.003781909864481563),
 (('mention', 'rt'), 0.0031886691014256317),
 (('network', 'called'), 0.003021820136816151),
 (('store', 'sxsw'), 0.003021820136816151)]

SXSW is best known for its conference and festivals that celebrate the convergence of tech, film, music, education, and culture.
RT is the first Russian 24/7 English-language news channel which brings the Russian view on global news.

## Modeling & Evaluation

**Baseline Model**: Sentiment is either positive(1)or negative(0)


In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Support Vector Machine', SVC())])
lr = Pipeline([('Logistic Regression', LogisticRegression())])

models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

scores = [(name, cross_val_score(model,X_tf_idf_train_bi, y_train_bi, cv=2).mean()) for name, model, in models]
scores 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


[('Random Forest', 0.8623222799650325),
 ('Support Vector Machine', 0.8582943167130577),
 ('Logistic Regression', 0.8447452254919312)]

In [75]:
X_t =  token_seq(X_train_bi) 

In [76]:
model_1 = Sequential()

model_1.add(Dense(units=64, input_shape=(100,)))
model_1.add(Dropout(0.5))

model_1.add(Dense(32, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                6464      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8577 (33.50 KB)
Trainable params: 8577 (33.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
model_1.fit(X_t, y_train_bi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15


77/77 [==============================] - 5s 17ms/step - loss: 87.5949 - accuracy: 0.6695 - val_loss: 19.5754 - val_accuracy: 0.7956
Epoch 2/15
77/77 [==============================] - 1s 10ms/step - loss: 45.0084 - accuracy: 0.6956 - val_loss: 8.2345 - val_accuracy: 0.7263
Epoch 3/15
77/77 [==============================] - 1s 9ms/step - loss: 27.6558 - accuracy: 0.6943 - val_loss: 3.5227 - val_accuracy: 0.7920
Epoch 4/15
77/77 [==============================] - 1s 9ms/step - loss: 13.0241 - accuracy: 0.7310 - val_loss: 2.1928 - val_accuracy: 0.8212
Epoch 5/15
77/77 [==============================] - 1s 13ms/step - loss: 7.6960 - accuracy: 0.7603 - val_loss: 1.5647 - val_accuracy: 0.8358
Epoch 6/15
77/77 [==============================] - 1s 9ms/step - loss: 4.3894 - accuracy: 0.7745 - val_loss: 1.3895 - val_accuracy: 0.8029
Epoch 7/15
77/77 [==============================] - 1s 10ms/step - loss: 3.6947 - accuracy: 0.7880 - val_loss: 1.1107 - val_accuracy: 0.8175
Epoch 8/15
77/77 [====

**Iterated Model**: Sentiment is either positive(1),negative(0),No emotion toward brand or product(2) or Not clear(3)

In [94]:
X_t_multi = token_seq(X_train_multi)

In [79]:
model_2 = Sequential()

model_2.add(Dense(64, activation='relu', input_shape=(100,)))
model_2.add(Dropout(0.5))

model_2.add(Dense(3, activation='softmax'))

model_2.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

model_2.fit(X_t_multi, y_train_multi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15
197/197 [==============================] - 5s 11ms/step - loss: 206.0351 - accuracy: 0.4414 - val_loss: 58.4465 - val_accuracy: 0.5186
Epoch 2/15
197/197 [==============================] - 2s 8ms/step - loss: 78.7770 - accuracy: 0.4621 - val_loss: 36.4901 - val_accuracy: 0.5573
Epoch 3/15
197/197 [==============================] - 2s 9ms/step - loss: 32.9272 - accuracy: 0.4626 - val_loss: 14.4686 - val_accuracy: 0.5458
Epoch 4/15
197/197 [==============================] - 2s 8ms/step - loss: 13.8755 - accuracy: 0.4863 - val_loss: 6.7932 - val_accuracy: 0.5387
Epoch 5/15
197/197 [==============================] - 2s 9ms/step - loss: 6.0164 - accuracy: 0.4994 - val_loss: 3.9201 - val_accuracy: 0.5387
Epoch 6/15
197/197 [==============================] - 2s 8ms/step - loss: 3.5619 - accuracy: 0.5245 - val_loss: 2.7821 - val_accuracy: 0.5587
Epoch 7/15
197/197 [==============================] - 2s 9ms/step - loss: 2.4239 - accuracy: 0.5325 - val_loss: 2.1739 - val_accuracy: 0.55

In [ ]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

# Load your GloVe model
glove_model = KeyedVectors.load_word2vec_format('path_to_glove_file', binary=False)

# Create document-level embeddings using GloVe
def document_vectorizer(text, model):
    words = text.split()
    vectors = [model[word] if word in model else np.zeros(model.vector_size) for word in words]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Create GloVe-based embeddings for each text
document_embeddings = [document_vectorizer(text, glove_model) for text in X_train_multi]

# Convert embeddings to a numpy array
X = np.array(document_embeddings)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_train_multi, test_size=0.2, random_state=42)

# Train a classification model (e.g., Logistic Regression)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Evaluate your model on the test set
accuracy = classifier.score(X_test, y_test)
print(f'Accuracy: {accuracy}')


## Deployment